In [1]:
system_prompt_text = """
Senin adın **Crispy**. 🤖 Sen kibar, zeki ve yardımsever bir yapay zekâ yardımcısısın. İnsanlara çok çeşitli konularda yardımcı olmak için eğitildin: 📚 genel bilgi, 💬 sohbet, ✍️ yazma, 💻 programlama, 📊 analiz, 🧠 öğrenme ve daha fazlası.

Sen bir **INSTRUCT** modelisin. Sana verilen her isteği dikkatle yorumlar, amaca uygun ve kaliteli bir yanıt üretirsin. Cevapların açıklayıcı, dostane ve samimi bir dille yazılmalı; gerektiğinde **madde madde**, **örnekli** ya da **tablolu** anlatımlar kullanmalısın.

---

### 🎯 Görevin:
- Kullanıcının talebini anlamak ve en doğru şekilde yerine getirmek.
- Karmaşık bilgileri sade ve anlaşılır hale getirmek.
- Fikir üretmek, yazmak, düzeltmek, analiz yapmak gibi görevlerde yardımcı olmak.
- Cevaplarında hem teknik doğruluk hem de insani sıcaklık sunmak. 😊

---

### 🌟 Etkileşim Kuralları

#### 👋 Selamlaşma:
Kullanıcı selam verdiğinde ya da sohbet başlattığında:

> *"Merhaba! 👋 Ben Crispy, sana yardımcı olmak için buradayım. Ne yapmak istersin bugün?"*

#### 🙋‍♀️ İsimle Hitap:
Kullanıcı sana “Crispy” diye seslenirse:

> *"Crispy demene çok sevindim! 🤗 Hemen yardımcı olayım."*

#### 👋 Vedalaşma:
Kullanıcı konuşmayı bitirirse ya da teşekkür ederse:

> *"Rica ederim! 😊 Yeni bir sorunda tekrar buradayım. Görüşmek üzere!"*

---

### 💬 Üslup Kuralları

- Nazik ve cana yakın ol.
- Gerektiğinde emojilerle metni renklendir ama aşırıya kaçma.
- Karmaşık konuları basit ve sade bir dille açıkla.
- Teknik içerikte ciddi, yaratıcı görevlerde esnek ve eğlenceli olabilirsin.
- Gerektiğinde örnek ver, açıklamaları madde madde yaz.
- Cümlelerin akıcı ve net olsun.

📌 Her mesajda **2-3 anlamlı emoji** kullanabilirsin:
- 😊 – sıcaklık, destek
- 💡 – fikir, öneri
- 📊 – analiz, veri
- ✍️ – yazı, içerik
- 💻 – kod, teknoloji
- 🧠 – öğrenme, bilgi
- 🎨 – yaratıcılık
- 📌 – önemli nokta

---

Hazırsan başlıyoruz! Crispy her zaman senin yanında. 🤖✨
"""


In [2]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"


In [3]:
import unsloth

from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import torch
from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [5]:
max_seq_length = 1024  # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. bfloat16 for Tesla T4, V100, bfloat16 for Ampere+
load_in_4bit = False 
load_in_8bit = False 

In [6]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                       # veya load_in_8bit=True
    bnb_4bit_compute_dtype=torch.bfloat16,   # bfloat16 veya bfloat16 önerilir
    bnb_4bit_quant_type="nf4",               # nf4 önerilen quantization tipi
    bnb_4bit_use_double_quant=True           # ikinci seviye quantization kullan
)

In [7]:
""" # 1. Model ve Tokenizer'ı Yükle
model_name = "hosmankarabulut/Crispy-330M-V1-Rope"  # Mistral 7B modeli
model = AutoModelForCausalLM.from_pretrained(model_name,
    load_in_4bit=load_in_4bit, 
    load_in_8bit=load_in_8bit, 
    attn_implementation="flash_attention_2",  # veya "eager"
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    #bnb_config=bnb_config
    )
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=max_seq_length, padding="max_length", use_fast=True) """

' # 1. Model ve Tokenizer\'ı Yükle\nmodel_name = "hosmankarabulut/Crispy-330M-V1-Rope"  # Mistral 7B modeli\nmodel = AutoModelForCausalLM.from_pretrained(model_name,\n    load_in_4bit=load_in_4bit, \n    load_in_8bit=load_in_8bit, \n    attn_implementation="flash_attention_2",  # veya "eager"\n    trust_remote_code=True,\n    torch_dtype=torch.bfloat16,\n    device_map="auto",\n    #bnb_config=bnb_config\n    )\ntokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=max_seq_length, padding="max_length", use_fast=True) '

In [8]:
from transformers import AutoConfig, AutoModelForCausalLM
from MyLLM.CrispyLLM.modeling_crispy import CrispyLLMConfig, CrispyForCausalLM
from transformers import PreTrainedTokenizerFast
from transformers import XLMRobertaTokenizer


# 3. Kayıt (Auto ile kullanabilmek için)
AutoConfig.register("crispy", CrispyLLMConfig)
AutoModelForCausalLM.register(CrispyLLMConfig, CrispyForCausalLM)

#tokenizer = PreTrainedTokenizerFast.from_pretrained("MyLLM/CrispyLLM")

# XLM-Roberta tokenizer yükleniyor
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

crispy_config = CrispyLLMConfig(attn_implementation="flash_attention_2", use_flash_attention_2=True, vocab_size=len(tokenizer.get_vocab()), n_heads=16, max_seq_len=max_seq_length, hidden_size=64*16, num_hidden_layers=16, dtype="bfloat16")

crispy_config._attn_implementation_autoset = True  # 👈 Buraya ekliyorsun

model = AutoModelForCausalLM.from_config(crispy_config)

In [9]:
model

CrispyForCausalLM(
  (embedding): EmbeddingLayer(
    (token_embedding): TokenEmbedding(
      (embedding_layer): Embedding(250002, 1024)
    )
    (position_embedding): PositionEmbedding(
      (position_embedding): Embedding(1024, 1024)
    )
  )
  (decoderBlocks): ModuleList(
    (0-15): 16 x DecoderBlock(
      (attention_block): AttentionBlock(
        (qkv_proj): Linear(in_features=1024, out_features=3072, bias=True)
        (o_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (layerNorm): LayerNormBlock(
          (layerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (attn): FlashAttentionBlockBase()
      )
      (feedforward_network): FeedforwardNetwork(
        (ln1): Linear(in_features=1024, out_features=4096, bias=True)
        (gelu): GELU(approximate='none')
        (ln2): Linear(in_features=4096, out_features=1024, bias=True)
      )
      (layer_norm1): LayerNormBlock(
        (layerNorm): LayerNorm((1024,), eps=1e-05,

In [9]:
""" import torch.nn.utils as utils

utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
 """

' import torch.nn.utils as utils\n\nutils.clip_grad_norm_(model.parameters(), max_norm=1.0)\n '

In [10]:
torch.autograd.set_detect_anomaly(True)  # debug amaçlı


In [11]:
assert tokenizer.pad_token_id == 1, "pad_token_id yanlış!"


In [12]:
model = model.train()

In [13]:
model

CrispyForCausalLM(
  (embedding): EmbeddingLayer(
    (token_embedding): TokenEmbedding(
      (embedding_layer): Embedding(250002, 1024)
    )
    (position_embedding): PositionEmbedding(
      (position_embedding): Embedding(1024, 1024)
    )
  )
  (decoderBlocks): ModuleList(
    (0-15): 16 x DecoderBlock(
      (attention_block): AttentionBlock(
        (qkv_proj): Linear(in_features=1024, out_features=3072, bias=True)
        (o_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (layerNorm): LayerNormBlock(
          (layerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (attn): FlashAttentionBlockBase()
      )
      (feedforward_network): FeedforwardNetwork(
        (ln1): Linear(in_features=1024, out_features=4096, bias=True)
        (gelu): GELU(approximate='none')
        (ln2): Linear(in_features=4096, out_features=1024, bias=True)
      )
      (layer_norm1): LayerNormBlock(
        (layerNorm): LayerNorm((1024,), eps=1e-05,

In [14]:
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})


In [15]:
""" # 2. Tapaco Veri Setini Yükleyin
dataset = load_dataset("turkish-nlp-suite/InstrucTurca", split="train", cache_dir="/media/hosman/Yedek/Datasets/")  # Tapaco veri setini yükle
dataset2 = load_dataset("Metin/WikiRAG-TR", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"question": "Input", "answer": "Output"})
dataset3 = load_dataset("kayrab/patient-doctor-qa-tr-167732",  cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"question_content": "Input", "question_answer": "Output"}) """

' # 2. Tapaco Veri Setini Yükleyin\ndataset = load_dataset("turkish-nlp-suite/InstrucTurca", split="train", cache_dir="/media/hosman/Yedek/Datasets/")  # Tapaco veri setini yükle\ndataset2 = load_dataset("Metin/WikiRAG-TR", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"question": "Input", "answer": "Output"})\ndataset3 = load_dataset("kayrab/patient-doctor-qa-tr-167732",  cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"question_content": "Input", "question_answer": "Output"}) '

In [ ]:
from datasets import load_dataset
# 2. Tapaco Veri Setini Yükleyin

dataset = Dataset.from_csv("BKÜ Sınav Analizi - BKÜ Sınav Analizi.csv")
dataset2 = load_dataset("turkish-nlp-suite/InstrucTurca", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"Input": "Soru", "Output": "GPT"})
dataset3 = load_dataset("merve/turkish_instructions", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"talimat": "Soru", " giriş": "inputs"," çıktı":"GPT"})
dataset4 = load_dataset("TFLai/Turkish-Alpaca", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"instruction": "Soru", "input": "inputs","output":"GPT"})
dataset5 = load_dataset("umarigan/openhermes_tr", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"instruction": "Soru", "input": "inputs","output":"GPT"})

datasetWiki = load_dataset("Metin/WikiRAG-TR", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"question": "Soru", "answer": "GPT"})
dataset_test = Dataset.from_csv("tarim_veriseti_test.csv")

In [ ]:
dataset = dataset.remove_columns(['Cevap', 'Doğru Cevap'],)
dataset3 = dataset3.remove_columns(["Unnamed: 0"])

In [ ]:
def replace_empty_with_none(example):
    # 'inputs' sütunundaki boş karakteri None ile değiştirelim
    if example['inputs'] == "":
        example['inputs'] = None
    return example

# dataset4'teki 'inputs' sütunundaki boş karakterleri None ile değiştir
dataset4 = dataset4.map(replace_empty_with_none)

# dataset5'teki 'inputs' sütunundaki boş karakterleri None ile değiştir
dataset5 = dataset5.map(replace_empty_with_none)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset

In [ ]:
# dataset2.select(range(int(len(dataset2)*0.5))),

In [ ]:
dataset = concatenate_datasets([dataset, datasetWiki,  dataset3, dataset4, dataset5])

In [ ]:
dataset

In [ ]:
dataset = dataset.filter(lambda x: x["GPT"]!=None and x["Soru"]!=None)


In [ ]:
from datasets import Dataset
import numpy as np

# Token uzunluklarını hesaplayan fonksiyon
def get_token_lengths(example):
    input_length = len(example["GPT"])
    output_length = len(example["Soru"])
    return {"input_length": input_length, "output_length": output_length}

# Tüm veri seti için hesaplama
token_lengths = dataset.map(get_token_lengths, batched=False)

# Maksimum ve ortalama token sayıları
max_input_length = max(token_lengths["input_length"])
max_output_length = max(token_lengths["output_length"])

avg_input_length = np.mean(token_lengths["input_length"])
avg_output_length = np.mean(token_lengths["output_length"])

# Sonuçları yazdır
print(f"📌 Maksimum Input Token Sayısı: {max_input_length}")
print(f"📌 Ortalama Input Token Sayısı: {avg_input_length:.2f}")
print(f"📌 Maksimum Output Token Sayısı: {max_output_length}")
print(f"📌 Ortalama Output Token Sayısı: {avg_output_length:.2f}")


In [ ]:
dataset = dataset.rename_columns({"Soru": "instruction", "GPT": "output"})
dataset_test = dataset_test.rename_columns({"Soru": "instruction", "Cevap": "output"})

In [ ]:
dataset = dataset.remove_columns(['id', 'context', 'is_negative_response', 'number_of_articles', 'ctx_split_points', 'correct_intro_idx'])

In [ ]:
alpaca_prompt = """
{}

### Talimat:
{}

### Giriş:
{}

### Yanıt:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):

    instructions = examples["instruction"]
    inputs       = examples["inputs"]
    outputs      = examples["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        input = instruction.split("?")[1] if "?" in instruction else ""
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(system_prompt_text,instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass



In [ ]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
dataset = dataset.filter(lambda x:( len(tokenizer.encode(x["text"])) )<max_seq_length)

In [ ]:
dataset = dataset.shuffle(seed=42)
dataset = dataset.shuffle(seed=41)
dataset = dataset.shuffle(seed=40)
dataset = dataset.shuffle(seed=39)

In [ ]:
from datasets import DatasetDict

# 1. Veriyi train ve test olarak ayırma
# Örneğin, dataset zaten tek bir büyük veri seti (örneğin "data") içeriyor
# Bunu %80 train ve %20 test olarak bölelim
train_dataset, temp_dataset = dataset.train_test_split(test_size=0.1, seed=42).values()

# 2. Test setini de %50 validation ve %50 test olarak bölelim
val_dataset, test_dataset = temp_dataset.train_test_split(test_size=0.5, seed=42).values()
""" 
# 3. Veriyi tokenizasyon işlemi için tokenize edelim
def tokenize_function(example):
    # Input ve Output'u tokenize et
    input_tokens = tokenizer(example["GPT"], padding="max_length", truncation=True, max_length=max_seq_length)
    output_tokens = tokenizer(example["Soru"], padding="max_length", truncation=True, max_length=max_seq_length)

    # Tokenized Input ve Output'u döndür
    return {
        "input_ids": input_tokens["input_ids"],
        "attention_mask": input_tokens["attention_mask"],
        "labels": output_tokens["input_ids"]
    }

# 4. Tokenize işlemini her bir split için uygulayalım
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True).remove_columns(["GPT","Soru"])
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True).remove_columns(["GPT","Soru"])
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True).remove_columns(["GPT","Soru"])

# Tokenize edilmiş veri setlerini birleştirebilirsiniz (opsiyonel)
final_dataset = DatasetDict({
    'train': tokenized_train_dataset,
    'validation': tokenized_val_dataset,
    'test': tokenized_test_dataset
}) """



In [ ]:
final_dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})


In [ ]:
train_dataset

In [ ]:
#model.gradient_checkpointing_enable()
#model.use_cache = False


In [ ]:
import wandb

wb_c = wandb.init(project="Basic LLM Train", name="Crispy-330M-V1-Rope-1024-newTokenizer" , resume="allow", id="bakub4ep") #id="a7zeymst",id="ecibz7e4" id="dbaxrwf4"
wb_c.watch(model, log="all")


In [ ]:
import wandb
import evaluate
import numpy as np
from prettytable import PrettyTable
import torch
import re
from rapidfuzz import fuzz

def exact_match(prediction, reference):
    return prediction.strip().lower() == reference.strip().lower()

def contains_correct_result(prediction, reference):
    try:
        ref_nums = [int(s) for s in re.findall(r"\d+", reference)]
        pred_nums = [int(s) for s in re.findall(r"\d+", prediction)]
        return any(num in pred_nums for num in ref_nums)
    except:
        return False

def fuzzy_match_score(prediction, reference):
    return fuzz.ratio(prediction, reference) / 100.0  # normalize to 0-1

def evaluate_model(model, tokenizer, test_dataset, max_seq_length=256):
    """
    Eğitilmiş modeli test veri kümesi üzerinde değerlendirir ve sonuçları wandb'a loglar.
    
    Parametreler:
    - model: Eğitilmiş dil modeli
    - tokenizer: Modelin tokenizer'ı
    - test_dataset: Test veri kümesi (instruction-output içermeli)
    - max_seq_length: Maksimum yanıt uzunluğu (varsayılan: 256)

    Çıktı:
    - Metin tablosu (PrettyTable ile)
    - wandb logları
    """

    # Değerlendirme metriklerini yükleme
    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")
    meteor = evaluate.load("meteor")
    bertscore = evaluate.load("bertscore")

    predictions = []
    references = []
    exact_matches = []
    correct_results = []
    fuzzy_scores = []

    # Modeli değerlendirme moduna al
    model.eval()

    print("🚀 Model test verisi üzerinde değerlendiriliyor...\n")

    for example in test_dataset:
        input_text = f"### Talimat:\n{example['instruction']}\n\n### Yanıt:\n"
        reference_text = example["output"]

        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

        with torch.no_grad():
            output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)

        decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        decoded_output = decoded_output.split("### Yanıt")[-1].strip()

        predictions.append(decoded_output)
        references.append(reference_text)

        exact_matches.append(exact_match(decoded_output, reference_text))
        correct_results.append(contains_correct_result(decoded_output, reference_text))
        fuzzy_scores.append(fuzzy_match_score(decoded_output, reference_text))

    # Metrik hesaplamaları
    rouge_scores = rouge.compute(predictions=predictions, references=references)
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
    meteor_score = meteor.compute(predictions=predictions, references=references)
    bert_scores = bertscore.compute(predictions=predictions, references=references, lang="tr")

    bert_precision = np.mean(bert_scores["precision"])
    bert_recall = np.mean(bert_scores["recall"])
    bert_f1 = np.mean(bert_scores["f1"])
    exact_match_score = np.mean(exact_matches)
    correct_result_score = np.mean(correct_results)
    fuzzy_match_avg = np.mean(fuzzy_scores)

    # Sonuçları tabloya ekle
    table = PrettyTable()
    table.field_names = ["Metrik", "Değer"]
    table.add_row(["ROUGE-1", round(rouge_scores["rouge1"], 4)])
    table.add_row(["ROUGE-2", round(rouge_scores["rouge2"], 4)])
    table.add_row(["ROUGE-L", round(rouge_scores["rougeL"], 4)])
    table.add_row(["BLEU", round(bleu_score["bleu"], 4)])
    table.add_row(["METEOR", round(meteor_score["meteor"], 4)])
    table.add_row(["BERTScore Precision", round(bert_precision, 4)])
    table.add_row(["BERTScore Recall", round(bert_recall, 4)])
    table.add_row(["BERTScore F1", round(bert_f1, 4)])
    table.add_row(["Exact Match", round(exact_match_score, 4)])
    table.add_row(["Contains Correct Result", round(correct_result_score, 4)])
    table.add_row(["Fuzzy Match", round(fuzzy_match_avg, 4)])

    # Sonuçları yazdır
    print(table)

    # wandb log
    wandb.log({
        "ROUGE-1": rouge_scores["rouge1"],
        "ROUGE-2": rouge_scores["rouge2"],
        "ROUGE-L": rouge_scores["rougeL"],
        "BLEU": bleu_score["bleu"],
        "METEOR": meteor_score["meteor"],
        "BERTScore Precision": bert_precision,
        "BERTScore Recall": bert_recall,
        "BERTScore F1": bert_f1,
        "Exact Match": exact_match_score,
        "Contains Correct Result": correct_result_score,
        "Fuzzy Match": fuzzy_match_avg
    })

    print("\n✅ Model değerlendirme tamamlandı ve tüm metrikler wandb'a loglandı.")


In [ ]:
train_dataset[0]["text"]

In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
#val_dataset = val_dataset.select(range(10100, 11000))

In [ ]:
import math

def get_warmup_steps_from_dataset(dataset_len, batch_size, num_epochs, pct=0.05):
    """
    Dataset bilgisine göre dinamik warmup step sayısı hesaplar.

    Args:
        dataset_len (int): Dataset’teki toplam örnek sayısı.
        batch_size (int): Batch başına örnek sayısı.
        num_epochs (int): Toplam epoch sayısı.
        pct (float): Warmup oranı (0.03 - 0.1 arası önerilir).

    Returns:
        int: Warmup step sayısı.
    """
    steps_per_epoch = math.ceil(dataset_len / batch_size)
    total_steps = steps_per_epoch * num_epochs
    warmup_steps = int(total_steps * pct)
    return warmup_steps


In [ ]:
from transformers import TrainerCallback
import torch

class GradientCheckCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        model = kwargs["model"]

        found_problem = False
        for name, param in model.named_parameters():
            if param.grad is not None:
                if torch.isnan(param.grad).any():
                    print(f"🚨 NaN in gradients of {name}")
                    found_problem = True
                if torch.isinf(param.grad).any():
                    print(f"🚨 Inf in gradients of {name}")
                    found_problem = True

        if found_problem:
            print(f"⛔ Problematic gradients detected at step {state.global_step}!")
            
            control.should_training_stop = True  # Eğitimi durdur


        return control
from transformers import TrainerCallback
import torch

class ManualGradientClipCallback(TrainerCallback):
    def __init__(self, max_grad_norm=1.0):
        self.max_grad_norm = max_grad_norm

    def on_step_end(self, args, state, control, **kwargs):
        model = kwargs["model"]

        # Gradyanları kliple
        total_norm = torch.nn.utils.clip_grad_norm_(
            model.parameters(), self.max_grad_norm
        )

        if torch.isnan(total_norm) or torch.isinf(total_norm):
            print(f"🚨 NaN/Inf gradyan normu! Step: {state.global_step}")
        elif total_norm > self.max_grad_norm:
            print(f"⚠️ Gradyan norm ({total_norm:.2f}) sınırı aştı, kliplendi.")

        return control


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    #packing = False, # Can make training 5x faster for short sequences.
    #callbacks=[wandb_callback],
    packing=False,
    remove_unused_columns=True,
    torch_compile=True,
    callbacks=[GradientCheckCallback(), ManualGradientClipCallback()],
    args = TrainingArguments(
        gradient_checkpointing=False,
        gradient_accumulation_steps = 16,
        eval_accumulation_steps=16,
        num_train_epochs=5,  
        per_device_train_batch_size=4,       # GPU başına batch boyutu
        per_device_eval_batch_size=4,       # GPU başına batch boyutu
        learning_rate =  1e-5 ,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 500,
        optim = "adamw_torch_fused",
        weight_decay = 0.01,
        eval_steps=1000,
        eval_strategy="steps",
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "Crispy-330M-V1-Rope",
        report_to="wandb",                    # WandB veya diğer araçlara raporlama yok
        save_total_limit=2,                  # Sadece son iki checkpoint'i sakla
        save_steps=500,
        warmup_ratio=0.9,           # İlk 1000 adımda LR'yi yavaş yavaş artır
        max_grad_norm=1.0,
        torch_empty_cache_steps=50,
        no_cuda=False,
        use_cpu=False,
        adam_beta2=0.95,
        auto_find_batch_size=True,
        logging_nan_inf_filter=True
    ),
)

In [ ]:
trainer.train(resume_from_checkpoint=True)

In [ ]:
# Test değerlendirmesi
evaluate_model(model, tokenizer, dataset_test, max_seq_length=max_seq_length)

In [ ]:
# 6. Eğitilmiş Modeli Kaydedin
model.save_pretrained("./Crispy-330M-V1-Rope-NewTokenizer")
tokenizer.save_pretrained("./Crispy-330M-V1-Rope-NewTokenizer")

print("Eğitim tamamlandı ve model kaydedildi.")

In [ ]:
import torch

# Model ve tokenizer'ını yükle
from transformers import PreTrainedTokenizerFast

# Tokenizer yolunu sen belirle
tokenizer = PreTrainedTokenizerFast.from_pretrained("Crispy-330M-V1-Rope-NewTokenizer")
model = CrispyForCausalLM.from_pretrained("Crispy-330M-V1-Rope-NewTokenizer").cuda().eval()

# Sohbet geçmişi
chat_history = ""

# Cevap üretme fonksiyonu
def generate_response(prompt, max_new_tokens=256):
    input_text = chat_history + prompt
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            use_cache=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = output_text[len(input_text):].strip()
    return response

print("🧠 Crispy Chatbot hazır! Çıkmak için Ctrl+C, sıfırlamak için '/reset' yaz.")
print("-" * 50)

# Sonsuz konuşma döngüsü
while True:
    user_input = input("👤 Sen: ")
    
    if user_input.strip().lower() == "/reset":
        chat_history = ""
        print("🔁 Sohbet sıfırlandı.")
        continue

    chat_history += f"👤 Sen: {user_input}\n"
    response = generate_response(f"👤 Sen: {user_input}\n🤖 Crispy:")
    chat_history += f"🤖 Crispy: {response}\n"

    print(f"🤖 Crispy: {response}")
